In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import h5py
import scipy.io as sio
import skimage.io

from Preprocess_Utilities import *

/home/admin-u6142160/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# data path
route_path = "../Data/090085/"
road_type = np.array(["motorway", "trunk", "primary", "secondary", "tertiary", "unclassified", "track", # 0-6
                      "residential", "service", "road", "living_street", # 7-10
                      "all_roads"]) # 11 
#                       "motor_trunk_pri_sec_tert_uncl_track", "motor_trunk_pri_sec_tert_uncl"]) # 12-13

path_raw_image = route_path + "090085_20170531.h5"
path_road_mask = np.char.add(np.char.add(np.char.add(route_path+"Road_Data/",road_type),
                                         np.char.add('/', road_type)), '.tif')

In [3]:
# read in raw image
raw_image = np.array(h5py.File(path_raw_image)['scene'])

# read in road mask
road_img_list = []
cnt = 0
for cur_path in path_road_mask:
    print(cnt, cur_path.split('/')[-1])
    cnt += 1
    road_img_list.append(skimage.io.imread(cur_path))

road_img_list = np.array(road_img_list)

# assert 0-1 coding
assert (np.logical_or(road_img_list == 1, road_img_list == 0)).all()

0 motorway.tif
1 trunk.tif
2 primary.tif
3 secondary.tif
4 tertiary.tif
5 unclassified.tif
6 track.tif
7 residential.tif
8 service.tif
9 road.tif
10 living_street.tif
11 all_roads.tif


In [ ]:
print(type(raw_image), raw_image.shape)
print(type(road_img_list), road_img_list.shape)

In [ ]:
1. motorway, trunk, primary 0.97 0.002 \\
2. motorway, trunk, primary, secondary 0.92 0.59 \\
3. motorway, trunk, primary, secondary, tertiary 0.88 0.68 \\
4. motorway, trunk, primary, secondary, tertiary, unclassified 0.77  0.11 \\
5*. motorway, trunk, primary, secondary, tertiary, unclassified, track 0.66 0.28 \\
6. motorway, trunk, primary, secondary, tertiary, unclassified, track, residential, service, road, living street 0.65 0.27 \\
7. secondary, tertiary, unclassified, track 0.67  0.19 \\
8. tertiary, unclassified, track 0.69  0.14 \\
9. residential, service, road, living street 0.96 0.84 \\
10. motorway, trunk, primary, residential, service, road, living street 0.94 0.15 \\
11. motorway, trunk, primary, secondary, residential, service, road, living street 0.91 0.13 \\


In [4]:
# modify the road mask
print("Used labels:")
combined_road_mask = 0
for i in [0,1,2]:
    print(path_road_mask[i])
    combined_road_mask += road_img_list[i]
print(combined_road_mask.shape, (combined_road_mask > 1).any())

combined_road_mask[np.where(combined_road_mask > 1)] = 1
assert (np.logical_or(combined_road_mask == 1, combined_road_mask == 0)).all()

Used labels:
../Data/090085/Road_Data/motorway/motorway.tif
../Data/090085/Road_Data/trunk/trunk.tif
../Data/090085/Road_Data/primary/primary.tif
(7961, 8091) True


In [5]:
image_patch = []
road_patch = []
road_existence = []

for row_offset in [0, 7, 14, 21]:
    for col_offset in [0, 7, 14, 21]:
        cur_img_pch, cur_rd_pch, cur_rd_ex = create_labelled_patches(raw_image, combined_road_mask,
                                                                     row_offset=row_offset,
                                                                     column_offset=col_offset)
        image_patch.extend(cur_img_pch)
        road_patch.extend(cur_rd_pch)
        road_existence.extend(cur_rd_ex)

image_patch = np.array(image_patch)
road_patch = np.array(road_patch)
road_existence = np.array(road_existence)

print(image_patch.shape, road_patch.shape, road_existence.shape)
# 349031

(349031, 7, 28, 28) (349031, 28, 28) (349031,)


In [14]:
road_existence.shape[0]

349031

In [ ]:
# save the file
h5_path = "../Data/090085/Road_Data/motor_trunk_pri/patch_set.h5"
h5f = h5py.File(h5_path, 'w')
h5f.create_dataset(name='image_patch', shape=image_patch.shape, data=image_patch)
h5f.create_dataset(name='road_patch', shape=road_patch.shape, data=road_patch)
# h5f.create_dataset(name='road_existence', shape=road_existence.shape, data=road_existence)
h5f.close()

# h5f = h5py.File(h5_path, 'r')
# h5_image_patch = np.array(h5f['image_patch'])
# h5_road_patch = np.array(h5f['road_patch'])
# h5_road_existence = np.array(h5f['road_existence'])
# h5f.close()

In [ ]:
# display some img
# show_image_against_road(raw_image, combined_road_mask, size=-1, figsize=(150,150), save_path="combined_road_mask")

In [ ]:
print((h5_image_patch == image_patch).all(), 
      (h5_road_patch == road_patch).all(), 
      (h5_road_existence == road_existence).all())

In [ ]:
# plot out some patches
# [0-21450]
index = 1000

has_road = road_existence[index]
mask = road_patch[index]
patch = np.array([image_patch[index][3].T, image_patch[index][2].T, image_patch[index][1].T]).T

print(has_road, patch.max(), patch.min(), patch.shape, mask.shape)
patch = patch * 0.0001 * 5

# plot raw img
plt.imshow(patch)
plt.show()
plt.clf()

# plot road mask
plt.imshow(mask, cmap=plt.get_cmap('gray'))
plt.show()
plt.clf()